In [1]:
from durable.lang import *

with ruleset('testRS'):
    # 규칙 집합 testRS
    
    @when_all(m.subject == 'World') #조건부
    def say_hello(c):
        print('hello {0}'.format(c.m.subject)) #c -> 평가 대상
        
post('testRS', { 'subject' : 'World'}) #규칙 집합에 데이터 'Subject' : 'World' 전달

hello World


{'sid': '0', 'id': 'sid-0', '$s': 1}

In [2]:
with ruleset('animal'):
    @when_all(
    c.first << (m.predicate == 'eats') & (m.object == 'flies'), 
        (m.predicate == 'lives') & (m.object == 'water') & (m.subject == c.first.subject)
    )   
    def frog(c):
        c.assert_fact({'subject' : c.first.subject, 'predicate' : 'is', 'object' : 'frog'})
        #add fact
    
    @when_all(
    c.first << (m.predicate == 'eats') & (m.object == 'flies'), 
        (m.predicate == 'lives') & (m.object == 'land') & (m.subject == c.first.subject)
    )   
    def chameleon(c):
        c.assert_fact({'subject' : c.first.subject, 'predicate' : 'is', 'object' : 'chameleon'})
    
    @when_all((m.predicate == 'eats') & (m.object == 'worms'))   
    def bird(c):
        c.assert_fact({'subject' : c.m.subject, 'predicate' : 'is', 'object' : 'bird'})
      
    @when_all((m.predicate == 'is') & (m.object == 'frog'))   
    def green(c):
        c.assert_fact({'subject' : c.m.subject, 'predicate' : 'is', 'object' : 'green'})
        
    @when_all((m.predicate == 'is') & (m.object == 'chameleon'))   
    def green(c):
        c.assert_fact({'subject' : c.m.subject, 'predicate' : 'is', 'object' : 'grey'})
    
    @when_all((m.predicate == 'is') & (m.object == 'bird'))   
    def green(c):
        c.assert_fact({'subject' : c.m.subject, 'predicate' : 'is', 'object' : 'black'})
    
    @when_all(+m.subject)   
    def output(c):
        print('Fact : {0} {1} {2}'.format(c.m.subject, c.m.predicate, c.m.object))
        

In [5]:
assert_fact('animal', {'subject' : 'Kermit', 'predicate' : 'eats', 'object' : 'flies'})
assert_fact('animal', {'subject' : 'Kermit', 'predicate' : 'lives', 'object' : 'water'})
assert_fact('animal', {'subject' : 'Greedy', 'predicate' : 'eats', 'object' : 'flies'})
assert_fact('animal', {'subject' : 'Greedy', 'predicate' : 'lives', 'object' : 'land'})
assert_fact('animal', {'subject' : 'Tweety', 'predicate' : 'eats', 'object' : 'worms'})

MessageObservedException: {'subject': 'Kermit', 'predicate': 'eats', 'object': 'flies'}

In [4]:
with ruleset('risk'):
    @when_all(c.first << m.t == 'purchase', c.second << m.location != c.first.location)
    def fraud(c):
        print('이상거래탐 탐지 --> {0}, {1}'.format(c.first.location, c.second.location))

post('risk', {'t' : 'purchase', 'location' : 'US'})
post('risk', {'t' : 'purchase', 'location' : 'CA'})

이상거래탐 탐지 --> CA, US


{'sid': '0', 'id': 'sid-0', '$s': 1}

In [12]:
with ruleset('bookstore'):
    @when_all(+m.status)
    def event(c):
        print('bookstore-> Reference {0} status {1}'.format(c.m.reference, c.m.status))
        
    @when_all(+m.name)
    def fact(c):
        print('bookstore-> Added {0}'.format(c.m.name))
        
    @when_all(none(+m.name))
    def empty(c):
        print('bookstore-> No books')
        
assert_fact('bookstore', {
    
    'name' : 'The new book',
    'seller' : 'bookstore',
    'reference' : '75323',
    'price' : 500
})


try:
    assert_fact('bookstore', {
    
    'name' : 'The new book',
    'seller' : 'bookstore',
    'reference' : '75323',
    'price' : 500
    })
except BaseException as e:
    print('Error : {0}'.format(e.message))

bookstore-> Added The new book
Error : Message has already been observed: {"name": "The new book", "seller": "bookstore", "reference": "75323", "price": 500}


In [13]:
post('bookstore', {
    'reference' : '75323',
    'status' : 'Active'
})

retract_fact('bookstore', {
    
    'name' : 'The new book',
    'seller' : 'bookstore',
    'reference' : '75323',
    'price' : 500
})

bookstore-> Reference 75323 status Active
bookstore-> No books


{'sid': '0', 'id': 'sid-0', '$s': 1}